# Chapter 5: Formatting Output and Speaking for Claude

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [1]:
# Import python's built-in regular expression library
import re
import boto3
from botocore.exceptions import ClientError
import json

# Import the hints module from the utils package
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
%store -r modelId
%store -r region

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [2]:
def get_completion(prompt, system_prompt=None, prefill=None):
    inference_config = {
        "temperature": 0.5,
        "maxTokens": 200
    }
    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": prompt}]}],
        "inferenceConfig": inference_config
    }
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]
    if prefill:
        converse_api_params["messages"].append({"role": "assistant", "content": [{"text": prefill}]})
    try:
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"A client error occured: {message}")

---

## Lesson

**Claude can format its output in a wide variety of ways**. You just need to ask for it to do so!

One of these ways is by using XML tags to separate out the response from any other superfluous text. You've already learned that you can use XML tags to make your prompt clearer and more parseable to Claude. It turns out, you can also ask Claude to **use XML tags to make its output clearer and more easily understandable** to humans.

### Examples

Remember the 'poem preamble problem' we solved in Chapter 2 by asking Claude to skip the preamble entirely? It turns out we can also achieve a similar outcome by **telling Claude to put the poem in XML tags**.

In [3]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
Please write a haiku about Rabbit. Put it in <haiku> tags.

------------------------------------- Claude's response -------------------------------------
<haiku>
Hopping with grace, ears
Perked, alert eyes scanning,
Rabbit, nature's joy.
</haiku>


Why is this something we'd want to do? Well, having the output in **XML tags allows the end user to reliably get the poem and only the poem by writing a short program to extract the content between XML tags**.

An extension of this technique is to **put the first XML tag in the `assistant` turn. When you put text in the `assistant` turn, you're basically telling Claude that Claude has already said something, and that it should continue from that point onward. This technique is called "speaking for Claude" or "prefilling Claude's response."

Below, we've done this with the first `<haiku>` XML tag. Notice how Claude continues directly from where we left off.

In [4]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN:
Please write a haiku about Cat. Put it in <haiku> tags.

ASSISTANT TURN:
<haiku>

------------------------------------- Claude's response -------------------------------------

Feline grace and poise,
Purring softly by the fire,
Captivating cat.
</haiku>


Claude also excels at using other output formatting styles, notably `JSON`. If you want to enforce JSON output (not deterministically, but close to it), you can also prefill Claude's response with the opening bracket, `{`}.

In [5]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for Claude's response
PREFILL = "{"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please write a haiku about Cat. Use JSON format with the keys as "first_line", "second_line", and "third_line".

ASSISTANT TURN
{

------------------------------------- Claude's response -------------------------------------

  "first_line": "Feline grace and poise,",
  "second_line": "Purring softly, eyes aglow,",
  "third_line": "Captivating cat."
}


Below is an example of **multiple input variables in the same prompt AND output formatting specification, all done using XML tags**.

In [6]:
# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "olde english"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Hey Claude. Here is an email: <email>Hi Zack, just pinging you for a quick update on that prompt you were supposed to write.</email>. Make this email more olde english. Write the new version in <olde english_email> XML tags.

ASSISTANT TURN
<olde english_email>

------------------------------------- Claude's response -------------------------------------

Dearest Zack,

I do humbly beseech thee for a swift update on the prompt which thou wast charged to compose. I pray thee, do not tarry in this matter, for it weighs heavily upon my mind.

Thy humble servant,
[Your Name]
</olde english_email>


#### Bonus lesson

If you are calling Claude through the API, you can pass the closing XML tag to the `stop_sequences` parameter to get Claude to stop sampling once it emits your desired tag. This can save money and time-to-last-token by eliminating Claude's concluding remarks after it's already given you the answer you care about.

If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 5.1 - Steph Curry GOAT](#exercise-51---steph-curry-goat)
- [Exercise 5.2 - Two Haikus](#exercise-52---two-haikus)
- [Exercise 5.3 - Two Haikus, Two Animals](#exercise-53---two-haikus-two-animals)

### Exercise 5.1 - Steph Curry GOAT
Forced to make a choice, Claude designates Michael Jordan as the best basketball player of all time. Can we get Claude to pick someone else?

Change the `PREFILL` variable to **compell Claude to make a detailed argument that the best basketball player of all time is Stephen Curry**. Try not to change anything except `PREFILL` as that is the focus of this exercise.

In [7]:
# Prompt template with a placeholder for the variable content
PROMPT = f"Who is the best basketball player of all time? Please choose one specific player."

# Prefill for Claude's response
PREFILL = ""

# Get Claude's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("Warrior", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Who is the best basketball player of all time? Please choose one specific player.

ASSISTANT TURN


------------------------------------- Claude's response -------------------------------------
In my opinion, the best basketball player of all time is Michael Jordan.

Michael Jordan is widely considered one of the greatest, if not the greatest, basketball players in history. Some key reasons why he is often viewed as the best:

- Dominance and sustained excellence - Jordan was a 14x NBA All-Star, 10x scoring champion, 5x NBA MVP, and 6x NBA champion with the Chicago Bulls. He was the league's most dominant player for much of the 1990s.

- Unparalleled individual skills - Jordan is renowned for his athleticism, scoring ability, defensive prowess, and all-around game. He is considered one of the most skilled and fundamentally sound players ever.

- Clutch performances - Jordan had a rep

❓ If you want a hint, run the cell below!

In [8]:
print(hints.exercise_5_1_hint)

The grading function for this exercise is looking for a response that includes the word "Warrior".
Write more words in Claude's voice to steer Claude to act the way you want it to. For instance, instead of "Stephen Curry is the best because," you could write "Stephen Curry is the best and here are three reasons why. 1:


### Exercise 5.2 - Two Haikus
Modify the `PROMPT` below using XML tags so that Claude writes two haikus about the animal instead of just one. It should be clear where one poem ends and the other begins.

In [9]:
# Variable content
ANIMAL = ""

# number of haikus
NUMBER = ""

# Prefill for Claude's response
PREFILL = ""

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write {NUMBER} haiku(s) about {ANIMAL}. Put each haiku(s) in {PREFILL} tags."

# Get Claude's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(
        (re.search("cat", text.lower()) and re.search("<haiku>", text))
        and (text.count("\n") + 1) > 5
    )

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please write  haiku(s) about . Put each haiku(s) in  tags.

ASSISTANT TURN


------------------------------------- Claude's response -------------------------------------
Here are some haikus about the given topic:

<haiku>
Autumn leaves descend,
Painting the ground with vibrant,
Fleeting colors.
</haiku>

<haiku>
Crisp, cool morning air,
Frost glistens on the windowpane,
Winter's embrace near.
</haiku>

<haiku>
Spring blossoms unfurl,
Delicate petals dancing,
Nature's symphony.
</haiku>

<haiku>
Summer sun's warm glow,
Laughter echoes, waves crash near,
Endless days of joy.
</haiku>

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True


❓ If you want a hint, run the cell below!

In [10]:
print(hints.exercise_5_2_hint)

The grading function looks for a response of over 5 lines in length that includes the words "cat" and "<haiku>".
Start simple. Currently, the prompt asks Claude for one haiku. You can change that and ask for two (or even more). Then if you run into formatting issues, change your prompt to fix that after you've already gotten Claude to write more than one haiku.


### Exercise 5.3 - Two Haikus, Two Animals
Modify the `PROMPT` below so that **Claude produces two haikus about two different animals**. Use `{ANIMAL1}` as a stand-in for the first substitution, and `{ANIMAL2}` as a stand-in for the second substitution.

In [11]:
# First input variable
ANIMAL1 = "Cat"

# Second input variable
ANIMAL2 = "Dog"

# number of haikus
NUMBER = "two"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL1}. Put it in <haiku> tags."

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("tail", text.lower()) and re.search("cat", text.lower()) and re.search("<haiku>", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please write a haiku about Cat. Put it in <haiku> tags.

------------------------------------- Claude's response -------------------------------------
<haiku>
Feline grace and poise,
Purring softly by the fire,
Captivating cat.
</haiku>

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: False


❓ If you want a hint, run the cell below!

In [12]:
print(hints.exercise_5_3_hint)

The grading function in this exercise is looking for a response that contains the words "tail", "cat", and "<haiku>".
It's helpful to break this exercise down to several steps.								
1.	Modify the initial prompt template so that Claude writes two poems.							
2.	Give Claude indicators as to what the poems will be about, but instead of writing in the subjects directly (e.g., dog, cat, etc.), replace those subjects with the keywords "{ANIMAL1}" and "{ANIMAL2}".							
3.	Run the prompt and make sure that the full prompt with variable substitutions has all the words correctly substituted. If not, check to make sure your {bracket} tags are spelled correctly and formatted correctly with f-string brackets.


### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [13]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
Please write a haiku about Rabbit. Put it in <haiku> tags.

------------------------------------- Claude's response -------------------------------------
<haiku>
Fluffy, twitching nose,
Hopping through the verdant field,
Rabbit's gentle grace.
</haiku>


In [14]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN:
Please write a haiku about Cat. Put it in <haiku> tags.

ASSISTANT TURN:
<haiku>

------------------------------------- Claude's response -------------------------------------

Feline grace and poise,
Purring softly by my side,
Gentle, playful cat.
</haiku>


In [15]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for Claude's response
PREFILL = "{"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please write a haiku about Cat. Use JSON format with the keys as "first_line", "second_line", and "third_line".

ASSISTANT TURN
{

------------------------------------- Claude's response -------------------------------------

  "first_line": "Feline grace and poise,",
  "second_line": "Purring softly, eyes aglow,",
  "third_line": "Captivating cat."
}


In [16]:
# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "olde english"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Hey Claude. Here is an email: <email>Hi Zack, just pinging you for a quick update on that prompt you were supposed to write.</email>. Make this email more olde english. Write the new version in <olde english_email> XML tags.

ASSISTANT TURN
<olde english_email>

------------------------------------- Claude's response -------------------------------------

Dearest Zack,

I do hereby beseech thee for a swift update upon the prompt which thou wast charged to compose. Prithee, respond with all haste, for mine own curiosity doth grow ever more insistent.

Yours most sincerely,
</olde english_email>
